In [38]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [43]:
# Load the pre-trained emotion recognition model
emotion_model = load_model('Facial_Recognition_fine_tuned.h5')  # Replace with your model file

# Remove the last layer of the emotion model
emotion_model.layers.pop()


In [44]:
import os

# Directory containing your images
image_directory = "Face_emotions/train"

# Initialize empty lists for data and labels
data = []
labels = []

# Iterate through files in the directory
for filename in os.listdir(image_directory):
    if filename.endswith(".jpg"):  # Adjust the file extension as needed
        # Extract emotion label from the filename or your dataset structure
        emotion_label = extract_emotion_label(filename)  # You need to implement this function
        if emotion_label is not None:
            # Construct the full image path
            image_path = os.path.join(image_directory, filename)
            # Preprocess the image and append it to the data list
            img = preprocess_image(image_path, target_size=(224, 224))
            data.append(img)
            # Append the emotion label to the labels list
            labels.append(emotion_label)

# Convert data and labels to NumPy arrays
import numpy as np
data = np.array(data)
labels = np.array(labels)


In [45]:
# Load the Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [46]:
# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [47]:
# Open a video capture stream (usually the default camera)
cap = cv2.VideoCapture(0)

In [48]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = gray[y:y + h, x:x + w]

        # Resize the face image to match the input size of the emotion recognition model (224x224x3)
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_GRAY2BGR)  # Convert back to 3 channels
        face_roi = img_to_array(face_roi) / 255.0
        face_roi = np.expand_dims(face_roi, axis=0)

        # Predict the emotion
        emotion_pred = emotion_model.predict(face_roi)
        emotion_label = emotion_labels[np.argmax(emotion_pred)]

        # Draw a rectangle around the detected face and label the emotion
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with emotion labels
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break




1/1 [==============================] - 0s 102ms/step


1/1 [==============================] - 0s 105ms/step


1/1 [==============================] - 0s 123ms/step


KeyboardInterrupt: 

In [49]:
# Release the video capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()